<a href="https://colab.research.google.com/github/zyz314/100-Days-Of-ML-Code/blob/master/project6_4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# testproject.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

import util
from util import get_data, plot_data

import indicators as ind
from indicators import work_all_indicators, plot_indicators

import marketsimcode as mc
from marketsimcode import market_simulator, generate_statistics

import TheoreticallyOptimalStrategy as tos
from TheoreticallyOptimalStrategy import testPolicy, get_stock_data


def author():
    return "yzheng438"


def calculate_statistics(portfolio_values, benchmark_values):
    """
    计算统计数据
    :param portfolio_values: 投资组合价值 DataFrame
    :param benchmark_values: 基准价值 DataFrame
    :return: 各种统计数据
    """
    # 归一化数据
    portfolio_normalized = portfolio_values / portfolio_values.iloc[0]
    benchmark_normalized = benchmark_values / benchmark_values.iloc[0]

    # 计算累计回报
    portfolio_cumulative_return = (portfolio_normalized.iloc[-1] / portfolio_normalized.iloc[0]) - 1
    benchmark_cumulative_return = (benchmark_normalized.iloc[-1] / benchmark_normalized.iloc[0]) - 1

    # 计算每日回报
    portfolio_daily_returns = portfolio_normalized.pct_change().dropna()
    benchmark_daily_returns = benchmark_normalized.pct_change().dropna()

    # 计算平均每日回报
    portfolio_mean_daily_return = portfolio_daily_returns.mean()
    benchmark_mean_daily_return = benchmark_daily_returns.mean()

    # 计算每日回报的标准差
    portfolio_std_daily_return = portfolio_daily_returns.std()
    benchmark_std_daily_return = benchmark_daily_returns.std()

    # 计算夏普比率
    portfolio_sharpe_ratio = (portfolio_mean_daily_return / portfolio_std_daily_return) * np.sqrt(252)
    benchmark_sharpe_ratio = (benchmark_mean_daily_return / benchmark_std_daily_return) * np.sqrt(252)

    return {
        "portfolio_cumulative_return": portfolio_cumulative_return,
        "benchmark_cumulative_return": benchmark_cumulative_return,
        "portfolio_mean_daily_return": portfolio_mean_daily_return,
        "benchmark_mean_daily_return": benchmark_mean_daily_return,
        "portfolio_std_daily_return": portfolio_std_daily_return,
        "benchmark_std_daily_return": benchmark_std_daily_return,
        "portfolio_sharpe_ratio": portfolio_sharpe_ratio,
        "benchmark_sharpe_ratio": benchmark_sharpe_ratio
    }


def main():
    """
    主函数，执行整个项目流程
    """
    # 设置股票符号和日期范围
    symbol = 'JPM'
    start_date = dt.datetime(2008, 1, 1)
    end_date = dt.datetime(2009, 12, 31)

    # 获取股票数据
    stock_data = get_stock_data(symbol, start_date, end_date)

    # 运行所有技术指标并生成图表
    indicators = work_all_indicators(stock_data)
    plot_indicators(stock_data, indicators)

    # 计算理论上的最佳交易策略
    trades = testPolicy(symbol, start_date, end_date, start_value=100000)

    # 执行市场模拟
    portfolio_values = market_simulator(trades, initial_cash=100000)

    # 获取基准数据
    benchmark = get_data([symbol], pd.date_range(start_date, end_date))
    benchmark = benchmark[symbol] * 1000  # 假设买入1000股并持有

    # 计算统计数据
    statistics = calculate_statistics(portfolio_values['portfolio'], benchmark)

    # 打印统计数据
    print("Portfolio Cumulative Return:", statistics["portfolio_cumulative_return"])
    print("Benchmark Cumulative Return:", statistics["benchmark_cumulative_return"])
    print("Portfolio Mean Daily Return:", statistics["portfolio_mean_daily_return"])
    print("Benchmark Mean Daily Return:", statistics["benchmark_mean_daily_return"])
    print("Portfolio Standard Deviation of Daily Return:", statistics["portfolio_std_daily_return"])
    print("Benchmark Standard Deviation of Daily Return:", statistics["benchmark_std_daily_return"])
    print("Portfolio Sharpe Ratio:", statistics["portfolio_sharpe_ratio"])
    print("Benchmark Sharpe Ratio:", statistics["benchmark_sharpe_ratio"])

    # 生成并保存投资组合价值的图表
    plt.figure(figsize=(10, 6))
    plt.plot(portfolio_values.index, portfolio_values['portfolio'] / portfolio_values['portfolio'].iloc[0], label='Portfolio', color='blue')
    plt.plot(benchmark.index, benchmark / benchmark.iloc[0], label='Benchmark', color='red')
    plt.title('Portfolio vs. Benchmark')
    plt.xlabel('Date')
    plt.ylabel('Normalized Value')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig('Figure_6_Portfolio_vs_Benchmark.png')
    plt.close()


if __name__ == "__main__":
    main()
